# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


### 5.1 Installing packages:

In [1]:
#!pip install selenium 
# this library is to manipulate browser:
#!pip install webdriver-manager
#!pip install html5lib

In [2]:
#To control the layout and width within Notebook:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [3]:
# Import necessary modules from Selenium:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service #To configure the WebDriver service:
from selenium.webdriver.common.by import By #Provides methods for locating elements:
from selenium.webdriver.support.ui import Select #for interacting with dropdowns
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 
import numpy as np
import os
import time
import re
import unidecode
import time 

### 5.2  Initialize the Chrome driver

#### Function to select a specific type of electoral process 

Using a function to set up the Chrome WebDriver provides a modular and reusable way to configure and initialize your WebDriver instances

In [4]:
def setup_driver():
    #First, set the path to the ChromeDriver executable
    service = Service(executable_path='chromedriver-win64/chromedriver.exe')
    # Configure options for the Chrome browser
    options = webdriver.ChromeOptions()
    # Initialize the Chrome WebDriver with the specified service and options
    driver = webdriver.Chrome(service=service, options=options)
    # Maximize the browser window
    driver.maximize_window()
    # Return the configured WebDriver instance
    return driver

### 5.3 Navigating the page: Election

#### Function to navigate to a given URL

This function provides a clean interface for the main script or other functions to use.

In [5]:
def go_to_url(driver, url):
    # Navigate the WebDriver to the specified URL
    driver.get(url)

### 5.4 Construction of the loop and export of information in an Excel file"Elecciones_resultados"

#### Function to select a specific type of electoral process

In [6]:
def select_process_type(driver, tipo_proceso_xpath, tipo_xpath):
    # Click on the element specified by tipo_proceso_xpath to open the process type dropdown
    driver.find_element(By.XPATH, tipo_proceso_xpath).click()
    
    # Pause for 2 seconds to allow the dropdown to appear
    time.sleep(2)
    
    # Click on the element specified by tipo_xpath to select the desired process type
    driver.find_element(By.XPATH, tipo_xpath).click()
    # Pause for 2 seconds to ensure the selection is registered
    time.sleep(2)

#### Function to choose the presidential electoral process

In [7]:
def choose_presidential_process(driver, eleccion_tipo_xpath):
    # Click on the element specified by eleccion_tipo_xpath to choose the presidential process
    driver.find_element(By.XPATH, eleccion_tipo_xpath).click()
    # Pause for 2 seconds to allow for the selection to be processed
    time.sleep(2)

#### Function to get the options for the electoral process

In [8]:
def get_process_options(driver, elecciones_xpath):
    # Find the dropdown element using the specified XPath, then create a Select object to interact with the dropdown and finally return the options of the dropdown
    return Select(driver.find_element(By.XPATH, elecciones_xpath)).options

#### Function to restart the electoral process
In this code, the 'driver.get(url)' function navigates WebDriver to a specific URL. After this, 'time.sleep(2)' introduces a 2 second pause to allow the page to fully load. Subsequently, two functions, 'select_process_type' and 'Choose_presidential_process', are called with the parameters WebDriver and XPath, interacting with specific elements of the web page to choose the type of electoral process and presidential process, respectively. Another 'time.sleep(2)' follows each function call to allow the page to respond to the selections made.

In [9]:
def restart_process(driver, url, tipo_proceso_xpath, tipo_xpath, eleccion_tipo_xpath):
    # Load the initial URL to restart the electoral process
    driver.get(url)
    # Pause script execution for 2 seconds to ensure the page loads before proceeding
    time.sleep(2)
    # Call the 'select_process_type' function to choose the type of electoral process
    select_process_type(driver, tipo_proceso_xpath, tipo_xpath)
    # Call the 'choose_presidential_process' function to choose the presidential process
    choose_presidential_process(driver, eleccion_tipo_xpath)
    time.sleep(2)

#### Function to access a specific electoral process
In this code, the 'access_electoral_process' function uses the WebDriver to access an election process on a web page. Uses the 'WebDriverWait' library to wait until an element identified by the provided XPath (xpath_election) is visible on the page. Once visible, click on said element using 'driver.find_element(By.XPATH, eleccion_xpath).click()'. Then, introduce a 2-second pause '(time.sleep(2))' to allow the page to respond to the action taken.

In [10]:
def access_electoral_process(driver, eleccion_xpath):
    # Wait for up to 10 seconds until the specified element (identified by XPath) is visible on the page
    # This ensures that the page has loaded and the element is ready for interaction
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, eleccion_xpath)))
    # Once the element is visible, locate it using the provided XPath and simulate a click action
    driver.find_element(By.XPATH, eleccion_xpath).click()
    # Pause script execution for 2 seconds after clicking to allow the page to respond or load additional content
    time.sleep(2)

#### Function to go to data section
The 'go_to_data_section' function navigates to a specific section on a webpage. It utilizes the Selenium WebDriver to locate an HTML element using the provided XPath (ver_datos_xpath). Once the element is located, the function clicks on it, simulating a user interaction. The subsequent 'time.sleep(2)' introduces a brief pause, allowing the page to respond or load content, providing stability to the script. 

In [11]:
def go_to_data_section(driver, ver_datos_xpath):
    # Locate the HTML element using the provided XPath that represents the 'VER DATOS' button or section
    # and simulate a click action to navigate to the data section
    driver.find_element(By.XPATH, ver_datos_xpath).click()
    # Pause script execution for 2 seconds after clicking to allow the page to respond or load additional content
    time.sleep(2)

#### Function to navigate to the 'CANDIDATOS Y RESULTADOS' section
The 'go_to_candidates_results' function is designed to facilitate navigation to a specific section on a webpage. Leveraging the Selenium WebDriver, it locates an HTML element identified by the provided XPath (candidatos_resultados_xpath). Subsequently, the function simulates a user click on this element, triggering an action, such as opening a submenu or navigating to a new page. The subsequent 'time.sleep(2)' introduces a short delay, allowing the page to respond or load content.

In [12]:
def go_to_candidates_results(driver, candidatos_resultados_xpath):
    # Locate the HTML element using the provided XPath that represents the button or section related to candidates results
    # and simulate a click action to navigate to the candidates results section
    driver.find_element(By.XPATH, candidatos_resultados_xpath).click()
    # Pause script execution for 2 seconds after clicking to allow the page to respond or load additional content
    time.sleep(2)

#### Function to extract data from the table on the webpage
The 'extract_data_from_table' function is responsible for retrieving tabular data from a specified section of a webpage. It uses the Selenium WebDriver to locate an HTML element identified by the provided XPath (tabla_resultado_xpath). Once the element is located, the function captures its inner HTML content, and subsequently, the Pandas library's read_html function is employed to parse this HTML and extract tabular data. The function returns a Pandas DataFrame containing the extracted data, providing a convenient way to work with structured information from the webpage

In [13]:
def extract_data_from_table(driver, tabla_resultado_xpath):
    # Obtain the table element using the provided XPath and the Selenium WebDriver instance
    # Extract the inner HTML content of the table element
    tabla_resultado = driver.find_element(By.XPATH, tabla_resultado_xpath)
    return pd.read_html(tabla_resultado.get_attribute('innerHTML'))[0]

#### Main function to run the entire process
The main function orchestrates a series of actions using Selenium and Pandas to extract and organize electoral process data from the specified webpage. Here's a breakdown of the steps:

In [14]:
def main():
   # Initialize the Chrome WebDriver using the setup_driver function
    driver = setup_driver()
    
    # Navigate to the main URL of the electoral website
    url = 'https://infogob.jne.gob.pe/Eleccion'
    go_to_url(driver, url)
    
    # XPath expressions to locate and select the type of electoral process
    tipo_proceso_xpath = '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]'
    tipo_xpath = '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]'
    select_process_type(driver, tipo_proceso_xpath, tipo_xpath)
    
    # XPath expression to choose the presidential process
    eleccion_tipo_xpath = '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]'
    choose_presidential_process(driver, eleccion_tipo_xpath)

    # XPath expression to locate the dropdown containing electoral process options
    elecciones_xpath = '//*[@id="IdEleccion"]'

    # Retrieve the available electoral process options
    procesos = get_process_options(driver, elecciones_xpath)

    # Create an empty DataFrame to store the extracted data   
    data = pd.DataFrame(columns=['Elecciones', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS'])

    # Loop through each electoral process
    for i in range(2, len(procesos) + 1):
        
        #Restart the electoral process
        restart_process(driver, url, tipo_proceso_xpath, tipo_xpath, eleccion_tipo_xpath)
    
        # XPath expression to locate the electoral process option    
        eleccion_xpath = f'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{i}]'
        # Get the name of the electoral process
        electoral = driver.find_element(By.XPATH, eleccion_xpath).text
        print(electoral)

        # Access the details of the electoral process  
        access_electoral_process(driver, eleccion_xpath)

        # XPath expression to locate the button for viewing data
        ver_datos_xpath = '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button'    
        # Navigate to the 'VER DATOS' section
        go_to_data_section(driver, ver_datos_xpath)

        # XPath expression to locate the menu item for candidates' results 
        candidatos_resultados_xpath = '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]'
        # Navigate to the 'CANDIDATOS Y RESULTADOS' section
        go_to_candidates_results(driver, candidatos_resultados_xpath)

        # XPath expression to locate the results table
        tabla_resultado_xpath = '//*[@id="CandidatosResultados"]/div/div[1]/div[2]/div[2]'
        # Extract data from the results table
        tabla_electoral = extract_data_from_table(driver, tabla_resultado_xpath)
        # Add the name of the electoral process as an identifier
        tabla_electoral['Elecciones'] = electoral
    
        # Append the extracted data to the main DataFrame
        data = pd.concat([data, tabla_electoral[['Elecciones', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]], ignore_index=True)
    
    # Print the final DataFrame containing all extracted data        
    print(data)

    # Export the resulting DataFrame to an Excel file without index
    data.to_excel('Elecciones_1931_to_2021.xlsx', index=False)

    # Close the WebDriver 
    driver.quit()

main()

PRESIDENCIAL 2021 - 2DA VUELTA
PRESIDENCIAL 2021
PRESIDENCIAL 2016 - 2DA VUELTA
PRESIDENCIAL 2016
PRESIDENCIAL 2011 - 2DA VUELTA
PRESIDENCIAL 2011
PRESIDENCIAL 2006 - 2DA VUELTA
PRESIDENCIAL 2006
PRESIDENCIAL 2001 - 2DA VUELTA
PRESIDENCIAL 2001
PRESIDENCIAL 2000 - 2DA VUELTA
PRESIDENCIAL 2000
PRESIDENCIAL 1995
PRESIDENCIAL 1990 - 2DA VUELTA
PRESIDENCIAL 1990
PRESIDENCIAL 1985
PRESIDENCIAL 1980
PRESIDENCIAL 1963
PRESIDENCIAL 1962
PRESIDENCIAL 1956
PRESIDENCIAL 1950
PRESIDENCIAL 1945
PRESIDENCIAL 1939
PRESIDENCIAL 1936
PRESIDENCIAL 1931
                         Elecciones                 ORGANIZACIÓN POLÍTICA  \
0    PRESIDENCIAL 2021 - 2DA VUELTA  PARTIDO POLITICO NACIONAL PERU LIBRE   
1    PRESIDENCIAL 2021 - 2DA VUELTA                        FUERZA POPULAR   
2    PRESIDENCIAL 2021 - 2DA VUELTA                       VOTOS EN BLANCO   
3    PRESIDENCIAL 2021 - 2DA VUELTA                           VOTOS NULOS   
4                 PRESIDENCIAL 2021  PARTIDO POLITICO NACIONAL PERU LIBRE 